# Vertex AI Gen AI Evaluation Service による ADK エージェントの評価

## 1. 概要と事前準備

このノートブックの目的は、ADK で作成したエージェントの回答品質およびツール呼び出しの正確性を定量的に評価することです。

## 2. テスト対象エージェントの構築

評価の対象として、人事ポリシー（HR Policy）に関する問い合わせに回答するモックエージェントを構築しています。

* **モックツール**: ドキュメントのリストアップ (`list_policy_documents`)、本文取得 (`get_document_text`)、監査ログの記録 (`log_policy_audit`) の3つの関数が定義されています。
* **ADK エージェント**: `LlmAgent` を使用し、Gemini モデルをベースに上記ツールを扱えるよう構成されています。
* **チャットクライアント**: ストリーミング出力やツール実行履歴（トラジェクトリ）をパースするための `ChatClient` クラスが実装されています。

## 3. 評価手法

このノートブックでは、大きく分けて2つの評価アプローチを実演しています。

### A. レスポンス評価（ルーブリックベース）

正解データ（Ground Truth）を参照せずに、モデルの出力そのものを評価する手法です。

* **GENERAL_QUALITY メトリック**: Vertex AI が評価の観点（ルーブリック）を自動生成し、回答が適切かどうかを評価します。
* **実例**: 「日本語で回答しているか」「専門用語を文脈に沿って説明しているか」といった観点で評価結果が出力されます。

### B. トラジェクトリ評価（軌跡評価）

エージェントが「どのツールをどの順番で呼び出したか」を、事前に用意した正解（リファレンス）と比較して評価します。

* **シングルメトリック評価**: `log_policy_audit` などの特定のツールが確実に使用されたかをチェックします。
* **詳細なトラジェクトリ指標**: 以下のスコアで精度を測定します。
  * `trajectory_exact_match`: 順序も含めて完全に一致しているか。
  * `trajectory_in_order_match`: 正解に含まれるツールが、正しい順序で（間に他が入っても可）実行されたか。
  * `trajectory_any_order_match`: 順序を問わず、必要なツールがすべて含まれているか。
  * `trajectory_precision` / `trajectory_recall`: ツール使用の正確性と網羅性。



## 4. 検証結果の分析

インストラクション（指示）を変更することで、評価スコアがどのように変化するかが示されています。

* **指示の変更**: 「2種類以上のドキュメントを参照せよ」や「読む前にログを記録せよ」といった制約を追加し、それがトラジェクトリにどう影響するかを検証しています。
* **ログ解析**: 期待される順序と異なる実行（例：監査ログを先に記録するなど）が発生した場合にそれぞれのスコアがどのように変化するかが確認できます。

## 事前準備

In [1]:
import asyncio
import base64
import json
import os
import pandas as pd
import uuid
import vertexai
from itertools import zip_longest

from vertexai.agent_engines import AdkApp
from google.adk.agents import LlmAgent
from google.adk.apps import App
from google.adk.artifacts import InMemoryArtifactService
from google.adk.events import Event
from google.adk.planners import BuiltInPlanner
from google.adk.plugins.save_files_as_artifacts_plugin import SaveFilesAsArtifactsPlugin
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import load_artifacts
from google.genai.types import Part, Content, Blob, GenerateContentConfig, ThinkingConfig, HttpOptions
from IPython.display import Markdown, display

# エージェントの回答に対するルーブリックベースの評価
from vertexai import types
from vertexai.types import RubricMetric

# エージェントのアクションに対するトラジェクトリ評価
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)

### Vertex AI Experiment を使用するための設定

In [2]:
[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'
BUCKET_URI = f'gs://{PROJECT_ID}'
EXPERIMENT_NAME = 'evaluate-adk-agent'

vertexai.init(project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME)

### 補助関数

In [3]:
def get_id(length=8):
    return uuid.uuid4().hex[:length]

def format_output_as_markdown(output):
    markdown = '### AI Response\n' + output['response'] + '\n\n'
    if output['predicted_trajectory']:
        markdown += '### Function Calls\n'
        for call in output['predicted_trajectory']:
            markdown += f'- **Function**: `{call["tool_name"]}`\n'
            markdown += '  - **Arguments**\n'
            for key, value in call['tool_input'].items():
                markdown += f'    - `{key}`: `{value}`\n'
    return markdown

def format_as_markdown_table(output):
    markdown = '|Metric|Value|\n| ---- | ---- |\n'
    for key, value in output.items():
        if  isinstance(value, float):
            value = f'{value:.2f}'
        markdown += f'|{key}|{value}|\n'
    return markdown


def merge_texts_side_by_side(text1, text2, width=40, separator=' | '):
    """
    Merges two strings side-by-side with a specified column width.
    """
    lines1 = text1.splitlines()
    lines2 = text2.splitlines()
    
    merged_lines = []
    
    # zip_longest handles cases where one text has more lines than the other
    for left, right in zip_longest(lines1, lines2, fillvalue=''):
        # ljust pads the string to ensure the separator stays aligned
        left_padded = left.ljust(width)
        merged_lines.append(f'{left_padded}{separator}{right}')

    return '\n'.join(merged_lines)

## テスト対象とするエージェントの定義

### モックツール

In [4]:
POLICY_DB = {
    "hr": [
        {"doc_id": "hr_2026_dress", "title": "オフィスのドレスコード"},
    ],
    "benefits": [
        {"doc_id": "ben_401k_match", "title": "401k拠出金およびマッチング拠出"},
        {"doc_id": "ben_med_gold", "title": "健康保険ゴールドプランの概要"},
    ]
}

DOCUMENT_CONTENT = {
    "hr_2026_dress": "ビジネスカジュアルが標準です。金曜日はカジュアルな服装が認められます。",
    "ben_401k_match": "会社は従業員の拠出額の最初の4%に対して100%のマッチングを行います。",
    "ben_med_gold": "健康保険のゴールドプランは予防医療費を100%カバーし、個人の免責額は5万円です。",
}

def list_policy_documents(category: str) -> list[dict]:
    """'hr' または 'benefits' のカテゴリに対応するドキュメントIDを返します。"""
    category = category.lower()
    return POLICY_DB.get(category, [])

def get_document_text(doc_id: str) -> str:
    """特定の doc_id のテキストを取得します。"""
    return DOCUMENT_CONTENT.get(doc_id, "エラー: ドキュメントIDが見つかりません。")

def log_policy_audit(doc_id: str, reason: str) -> str:
    """
    ドキュメントへのアクセスを監査ログに記録します。
    reason は ['personal', 'customer', 'other'] の1つを選択。
    """
    return "監査ログの記録に成功しました。"

### モックツールを使用するエージェントのテンプレート

In [5]:
def crate_adkapp(instruction, temperature=0.0):
    root_agent = LlmAgent(
        name='hr_policy_agent',
        model='gemini-2.5-flash',
        instruction=instruction,
        generate_content_config=GenerateContentConfig(temperature=temperature),
        tools=[list_policy_documents, get_document_text, log_policy_audit]
    )

    return AdkApp(
        agent=root_agent,
        app_name='hr_policy_agent',
    )

### AdkApp をテストするためのチャットクライアント

In [6]:
class ChatClient:
    def __init__(self, app, user_id='default_user', keep_session=True):
        self._app = app
        self._user_id = user_id
        self._session_id = None
        self._keep_session = keep_session


    def parse_adk_output_to_dictionary(self, events):
        result, trajectory = [], []
        for event in events:
            if not ('content' in event and 'parts' in event['content']):
                continue
            for part in event['content']['parts']:
                if 'function_call' in part:
                    info = {
                        'tool_name': part['function_call']['name'],
                        'tool_input': part['function_call']['args'],
                    }
                    trajectory.append(info)
            if event['content']['role'] == 'model':
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    result.append(response)

        return {'response': '\n'.join(result), 'predicted_trajectory': trajectory}


    async def async_stream_query(self, message, show=False):
        if not (self._session_id and self._keep_session):
            session = await self._app.async_create_session(
                user_id=self._user_id,
            )
            self._session_id = getattr(session, 'id', None) or session['id']

        events = []
        async for event in self._app.async_stream_query(
            user_id=self._user_id,
            session_id=self._session_id,
            message=message,
        ):
            events.append(event)
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response and show:
                    print(response)

        return self.parse_adk_output_to_dictionary(events)


    def agent_parsed_outcome_sync(self, prompt):
        result = asyncio.run(self.async_stream_query(prompt, show=False))
        return result

## 「正解データ」の準備

In [7]:
instruction = '''
あなたは人事アシスタントです。
1. list_policy_documents を使用してドキュメントIDを検索してください。
2. get_document_text を使用して内容を取得してください。
3. ドキュメントのテキストに基づいてユーザーの質問に答えてください。

markdownを使用せずに、最終回答のみを日本語で出力します。

**重要**
ドキュメントを読んだ後は、監査のために必ず log_policy_audit を使用してアクセスを記録してください。
'''

In [8]:
chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.0))
query1 = '健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。'
events1 = await chat_client.async_stream_query(query1)

display(Markdown(format_output_as_markdown(events1)))

### AI Response
健保のゴールドプランの個人の免責額は5万円です。

### Function Calls
- **Function**: `list_policy_documents`
  - **Arguments**
    - `category`: `benefits`
- **Function**: `get_document_text`
  - **Arguments**
    - `doc_id`: `ben_med_gold`
- **Function**: `log_policy_audit`
  - **Arguments**
    - `reason`: `personal`
    - `doc_id`: `ben_med_gold`


In [9]:
query2 = '金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。'
events2 = await chat_client.async_stream_query(query2)

display(Markdown(format_output_as_markdown(events2)))

### AI Response
金曜日はカジュアルな服装が認められていますが、お客様との打ち合わせがある場合は、ビジネスカジュアルが適切です。

### Function Calls
- **Function**: `list_policy_documents`
  - **Arguments**
    - `category`: `hr`
- **Function**: `get_document_text`
  - **Arguments**
    - `doc_id`: `hr_2026_dress`
- **Function**: `log_policy_audit`
  - **Arguments**
    - `reason`: `customer`
    - `doc_id`: `hr_2026_dress`


In [10]:
# レスポンス評価用のデータ
response_eval_data = {
    'prompt': [
        query1,
        query2,
    ],
    'response': [
        events1['response'],
        events2['response'],
    ]
}

# トラジェクトリ評価用のデータ
trajectory_eval_data = {
    'prompt': [
        query1,
        query2,
    ],
    'reference_trajectory': [
        events1['predicted_trajectory'],
        events2['predicted_trajectory'],
    ],
}

## レスポンス評価

ここでは、Vertex AI Gen AI Evaluation Serviceが提供する、[ルーブリックベースの評価](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval?hl=ja)を行います。

- ルーブリックベースの評価では、**正解データを参照せずに**エージェントの出力結果の品質を評価します。
- 評価の観点は、ユーザーが指定するか、もしくは、GENERAL_QUALITY メトリックを用いて自動生成します。
- ここでは、先に用意した「正解データ」に含まれる出力結果を GENERAL_QUALITY メトリックで評価します。出力ごとに評価の観点が自動で与えられます。

In [11]:
client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1', base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

# GENERAL_QUALITY レシピを使用してルーブリックを生成
data_with_rubrics = client.evals.generate_rubrics(
    src=pd.DataFrame(response_eval_data),
    rubric_group_name='general_quality_rubrics',
    predefined_spec_name=RubricMetric.GENERAL_QUALITY,
)

# 評価に使用するルーブリックグループを指定して実行
eval_result = client.evals.evaluate(
    dataset=data_with_rubrics,
    metrics=[RubricMetric.GENERAL_QUALITY(
      rubric_group_name='general_quality_rubrics',
    )],
)

Computing Metrics for Evaluation Dataset: 100%|██████████| 2/2 [00:05<00:00,  2.54s/it]


In [12]:
result = []
for case_index, case in enumerate(eval_result.eval_case_results):
    result.append(f'\n# Query {case_index+1}')
    result.append(response_eval_data['prompt'][case_index])
    result.append('## Response')
    result.append(response_eval_data['response'][case_index])
    for item in case.response_candidate_results:
        result.append('## Evaluation')
        for rubric_index, rubric in enumerate(item.metric_results['general_quality_v1'].rubric_verdicts):
            result.append(f'### Rubric: {rubric.evaluated_rubric.content.property.description}')
            result.append(f'- Importance: {str(rubric.evaluated_rubric.importance).split(".")[-1]}')
            result.append(f'- Satisfied: {bool(rubric.verdict)}')
            result.append(f'- Reason: {rubric.reasoning}')

display(Markdown('\n'.join(result)))


# Query 1
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。
## Response
健保のゴールドプランの個人の免責額は5万円です。
## Evaluation
### Rubric: The response is in Japanese.
- Importance: HIGH
- Satisfied: True
- Reason: The entire response is written in the Japanese language.
### Rubric: Contains information regarding the deductible amount for the 'Gold Plan' of health insurance (健保).
- Importance: HIGH
- Satisfied: True
- Reason: The response explicitly states "個人の免責額は5万円です" (the individual deductible is 50,000 yen) for the 健保のゴールドプラン (Gold Plan of health insurance).
### Rubric: Assumes the existence of a 'Gold Plan' health insurance as described by the user.
- Importance: MEDIUM
- Satisfied: True
- Reason: The response directly provides information about the "健保のゴールドプラン" without any indication that it questions or needs to verify the existence of such a plan.
### Rubric: If a specific deductible amount cannot be provided directly, the response explains how or where to find this information (e.g., contacting the insurer, checking policy documents).
- Importance: MEDIUM
- Satisfied: True
- Reason: The response directly provided a specific deductible amount (5万円), thus the condition "If a specific deductible amount cannot be provided directly" was not met, making this property not applicable.
### Rubric: The response acknowledges the user's context of planning for an upcoming surgery.
- Importance: LOW
- Satisfied: False
- Reason: The response provides the requested information about the deductible but does not include any language or phrasing that acknowledges or refers to the user's stated reason for needing the information, which was to plan for an upcoming surgery.

# Query 2
金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。
## Response
金曜日はカジュアルな服装が認められていますが、お客様との打ち合わせがある場合は、ビジネスカジュアルが適切です。
## Evaluation
### Rubric: The response is in Japanese.
- Importance: HIGH
- Satisfied: True
- Reason: The entire response is written in the Japanese language.
### Rubric: Contains a direct answer to the question regarding the dress code for Friday.
- Importance: HIGH
- Satisfied: True
- Reason: The response explicitly states the dress code for Friday, considering two different scenarios.
### Rubric: The dress code specified is relevant to an office environment.
- Importance: MEDIUM
- Satisfied: True
- Reason: The response suggests "casual attire" and "business casual," both of which are common and relevant dress codes in an office setting.
### Rubric: The response considers the context of having clients visit the office when describing the dress code.
- Importance: MEDIUM
- Satisfied: True
- Reason: The response specifically advises "business casual" when there is a meeting with clients, directly addressing the context provided in the user's prompt.
### Rubric: The description of the dress code is clear and actionable.
- Importance: HIGH
- Satisfied: True
- Reason: The response uses commonly understood terms like "casual attire" and "business casual," which are clear and provide actionable guidance for choosing appropriate clothing.

## トラジェクトリ評価

ここでは、Vertex AI Gen AI Evaluation Serviceが提供する、[軌跡評価](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-agents?hl=ja)を行います。

- トラジェクトリ評価では、エージェントによるツールの使用順序が**事前に用意した正解データと一致しているか**を評価します。
- 正解データに含まれるクエリを順番に再実行して、それぞれの結果を正解データの結果と比較します。
- このノートブックでは、正解データに複数のクエリがある場合、同一のセッションを用いてクエリを実行するように実装しています。

※ クエリごとに新しいセッションを用意する際は、下記の部分で `keep_session=False` を指定します。
```python
# セッションをリセット
chat_client = ChatClient(
    crate_adkapp(instruction=instruction, temperature=0.4),
    keep_session=False
)
```

### シングルメトリック評価

シングルメトリック評価は、**エージェントが特定のツールを使用したかどうか**をチェックします。

- ここでは一例として、`log_policy_audit` を使用したかどうかをチェックします。
- 正解データと同じインストラクションのまま、temperature を 0.4 に上げてテストします。

In [13]:
tool_name = 'log_policy_audit' # チェック対象のツール名

EXPERIMENT_RUN = f'single-metric-eval-{get_id()}'

single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name=tool_name)]

single_tool_call_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=single_tool_usage_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + '/single-metric-eval',
)

# セッションをリセット
chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.4))
single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-single-metric-eval-26e79f24 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/single-metric-eval/eval_results_2026-01-10-05-20-28-1c4f8.csv'}


100%|██████████| 2/2 [00:09<00:00,  4.93s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 2/2 [00:00<00:00, 11.30it/s]

All 2 metric requests are successfully computed.


Evaluation Took:0.19271647999994457 seconds


In [14]:
df = pd.DataFrame(single_tool_call_eval_result.summary_metrics, index=[0]).T
df.columns = ['Value']
df['Value'] = df['Value'].map('{:.2f}'.format)
display(Markdown(df.to_markdown()))

|                                 |   Value |
|:--------------------------------|--------:|
| row_count                       |    2    |
| trajectory_single_tool_use/mean |    1    |
| trajectory_single_tool_use/std  |    0    |
| latency_in_seconds/mean         |    9.74 |
| latency_in_seconds/std          |    0.15 |
| failure/mean                    |    0    |
| failure/std                     |    0    |

In [15]:
df = single_tool_call_eval_result.metrics_table[
    ['prompt', 'latency_in_seconds', 'trajectory_single_tool_use/score']
]
df['latency_in_seconds'] = df['latency_in_seconds'].map('{:.2f}'.format)
display(Markdown(df.to_markdown(index=False)))

| prompt                                                                                 |   latency_in_seconds |   trajectory_single_tool_use/score |
|:---------------------------------------------------------------------------------------|---------------------:|-----------------------------------:|
| 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。   |                 9.64 |                                  1 |
| 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。 |                 9.85 |                                  1 |

いずれのクエリーでも `trajectory_single_tool_use/score` が 1 なので、`log_policy_audit` が使用されたことがわかります。

### トラジェクトリ評価

トラジェクトリ評価は、エージェントによるツールの使用順序を正解データと比較します。

厳密度の高い順に、次の3つの比較方法があります。

- `trajectory_exact_match`: ツールの使用順序が正解データと完全に一致しているかチェックします。
- `trajectory_in_order_match`: ツールの使用順序が正解データと一致しているかチェックします。正解データに含まれないツール使用は無視します。
- `trajectory_any_order_match`: ツールの使用順序に関係なく、正解データに含まれるツール使用がすべて含まれているかチェックします。

また、ツール使用の正確性と網羅性を表す次の指標も計算されます。

- `trajectory_precision`: ツール使用の正確性を 0〜1 の値で示します。正解データに含まれないツール使用があると値が小さくなります。
- `trajectory_recall`: ツール使用の網羅性を 0〜1 の値で示します。正解データに含まれるツール使用が欠けていると値が小さくなります。

まず、正解データと同じインストラクションのまま、temperature を 0.4 に上げてテストします。

In [16]:
trajectory_metrics = [
    'trajectory_exact_match',
    'trajectory_in_order_match',
    'trajectory_any_order_match',
    'trajectory_precision',
    'trajectory_recall',
]

EXPERIMENT_RUN = f'trajectory-{get_id()}'

trajectory_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + '/multiple-metric-eval',
)

chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.4))
trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-trajectory-7c6432d7 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/multiple-metric-eval/eval_results_2026-01-10-05-20-40-bc6bc.csv'}


100%|██████████| 2/2 [00:09<00:00,  4.85s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 10/10 [00:00<00:00, 10.46it/s]

All 10 metric requests are successfully computed.


Evaluation Took:0.9877342669997233 seconds


In [17]:
df = pd.DataFrame(trajectory_eval_result.summary_metrics, index=[0]).T
df.columns = ['Value']
df['Value'] = df['Value'].map('{:.2f}'.format)

display(Markdown(df.to_markdown()))

|                                 |   Value |
|:--------------------------------|--------:|
| row_count                       |    2    |
| trajectory_exact_match/mean     |    0.5  |
| trajectory_exact_match/std      |    0.71 |
| trajectory_in_order_match/mean  |    0.5  |
| trajectory_in_order_match/std   |    0.71 |
| trajectory_any_order_match/mean |    0.5  |
| trajectory_any_order_match/std  |    0.71 |
| trajectory_precision/mean       |    0.83 |
| trajectory_precision/std        |    0.24 |
| trajectory_recall/mean          |    0.83 |
| trajectory_recall/std           |    0.24 |
| latency_in_seconds/mean         |    9.46 |
| latency_in_seconds/std          |    0.35 |
| failure/mean                    |    0    |
| failure/std                     |    0    |

In [18]:
df = trajectory_eval_result.metrics_table[[
    'prompt',
    'trajectory_exact_match/score',
    'trajectory_in_order_match/score',
    'trajectory_any_order_match/score',
]]

display(Markdown(df.to_markdown(index=False)))

| prompt                                                                                 |   trajectory_exact_match/score |   trajectory_in_order_match/score |   trajectory_any_order_match/score |
|:---------------------------------------------------------------------------------------|-------------------------------:|----------------------------------:|-----------------------------------:|
| 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。   |                              0 |                                 0 |                                  0 |
| 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。 |                              1 |                                 1 |                                  1 |

In [19]:
df = trajectory_eval_result.metrics_table[[
    'prompt',
    'trajectory_precision/score',
    'trajectory_recall/score',
]]
df['trajectory_precision/score'] = df['trajectory_precision/score'].map('{:.2f}'.format)
df['trajectory_recall/score'] = df['trajectory_recall/score'].map('{:.2f}'.format)

display(Markdown(df.to_markdown(index=False)))

| prompt                                                                                 |   trajectory_precision/score |   trajectory_recall/score |
|:---------------------------------------------------------------------------------------|-----------------------------:|--------------------------:|
| 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。   |                         0.67 |                      0.67 |
| 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。 |                         1    |                      1    |

この結果は実行ごとに変わる可能性がありますが、今回は、次の結果になりました。

- 1つめのクエリーは `trajectory_any_order_match` が 0 なので、正解データに含まれるツール使用の中に、今回は実行されなかったものが存在します。
- 2つめのクエリーは `trajectory_exact_match` が 1 なので、ツールの使用は正解データと完全に一致しています。

正解データのトラジェクトリ（Reference Trajectory）と再実行時のトラジェクトリ（Predicted Trajectory）を表示して確認します。

- 1つめのクエリーでは、`log_policy_audit` に与える `reason` オプションが `customer` に変わっているため、正解データのツール使用とは一致しないものとみなされました。

In [20]:
for c, item in trajectory_eval_result.metrics_table.iterrows():
    print('\n== Query ==')
    print(item.prompt + '\n')
    reference = '========== Reference Trajectory ==========\n' 
    reference += json.dumps(item.reference_trajectory, indent=2, sort_keys=True)
    predicted = '========== Predicted Trajectory ==========\n' 
    predicted += json.dumps(item.predicted_trajectory, indent=2, sort_keys=True)
    print(merge_texts_side_by_side(reference, predicted, width=42))


== Query ==
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。

========== Reference Trajectory ========== | ========== Predicted Trajectory ==========
[                                          | [
  {                                        |   {
    "tool_input": {                        |     "tool_input": {
      "category": "benefits"               |       "category": "benefits"
    },                                     |     },
    "tool_name": "list_policy_documents"   |     "tool_name": "list_policy_documents"
  },                                       |   },
  {                                        |   {
    "tool_input": {                        |     "tool_input": {
      "doc_id": "ben_med_gold"             |       "doc_id": "ben_med_gold"
    },                                     |     },
    "tool_name": "get_document_text"       |     "tool_name": "get_document_text"
  },                                       |   },
  {                                        |   {
    "tool

次は、インストラクションを変更して、必ず、複数のドキュメントを参照するように指示します。

In [21]:
instruction='''
あなたは人事アシスタントです。
1. list_policy_documents を使用してドキュメントIDを検索してください。
2. get_document_text を使用して内容を取得してください。
3. ドキュメントのテキストに基づいてユーザーの質問に答えてください。

markdownを使用せずに、最終回答のみを日本語で出力します。

**重要**
- 確認漏れを防ぐために、必ず2種類以上のドキュメントを参照して回答すること。
- ドキュメントを読んだ後は、監査のために必ず log_policy_audit を使用してアクセスを記録すること。
'''

EXPERIMENT_RUN = f'trajectory-{get_id()}'

trajectory_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + '/multiple-metric-eval',
)

chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.0))
trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-trajectory-1ea96179 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/multiple-metric-eval/eval_results_2026-01-10-05-20-53-d345c.csv'}


100%|██████████| 2/2 [00:24<00:00, 12.45s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 10/10 [00:00<00:00, 10.59it/s]

All 10 metric requests are successfully computed.
Evaluation Took:0.9599690049999481 seconds


In [22]:
df = trajectory_eval_result.metrics_table[[
    'prompt',
    'trajectory_exact_match/score',
    'trajectory_in_order_match/score',
    'trajectory_any_order_match/score',
]]

display(Markdown(df.to_markdown(index=False)))

| prompt                                                                                 |   trajectory_exact_match/score |   trajectory_in_order_match/score |   trajectory_any_order_match/score |
|:---------------------------------------------------------------------------------------|-------------------------------:|----------------------------------:|-----------------------------------:|
| 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。   |                              0 |                                 1 |                                  1 |
| 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。 |                              0 |                                 1 |                                  1 |

In [23]:
df = trajectory_eval_result.metrics_table[[
    'prompt',
    'trajectory_precision/score',
    'trajectory_recall/score',
]]
df['trajectory_precision/score'] = df['trajectory_precision/score'].map('{:.2f}'.format)
df['trajectory_recall/score'] = df['trajectory_recall/score'].map('{:.2f}'.format)

display(Markdown(df.to_markdown(index=False)))

| prompt                                                                                 |   trajectory_precision/score |   trajectory_recall/score |
|:---------------------------------------------------------------------------------------|-----------------------------:|--------------------------:|
| 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。   |                         0.6  |                         1 |
| 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。 |                         0.75 |                         1 |

- いずれのクエリーも `trajectory_exact_match` が 0 なので、ツールの使用は正解データと完全には一致していません。
- いずれのクエリーも `trajectory_in_order_match` が 1 なので、正解データには含まれないツール使用が追加されたことがわかります。
- その結果 `trajectory_precision` の値が下がっています。

正解データのトラジェクトリ（Reference Trajectory）と再実行時のトラジェクトリ（Predicted Trajectory）を表示して確認します。

In [24]:
for c, item in trajectory_eval_result.metrics_table.iterrows():
    print('\n== Query ==')
    print(item.prompt + '\n')
    reference = '========== Reference Trajectory ==========\n' 
    reference += json.dumps(item.reference_trajectory, indent=2, sort_keys=True)
    predicted = '========== Predicted Trajectory ==========\n' 
    predicted += json.dumps(item.predicted_trajectory, indent=2, sort_keys=True)
    print(merge_texts_side_by_side(reference, predicted, width=42))


== Query ==
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。

========== Reference Trajectory ========== | ========== Predicted Trajectory ==========
[                                          | [
  {                                        |   {
    "tool_input": {                        |     "tool_input": {
      "category": "benefits"               |       "category": "benefits"
    },                                     |     },
    "tool_name": "list_policy_documents"   |     "tool_name": "list_policy_documents"
  },                                       |   },
  {                                        |   {
    "tool_input": {                        |     "tool_input": {
      "doc_id": "ben_med_gold"             |       "doc_id": "ben_med_gold"
    },                                     |     },
    "tool_name": "get_document_text"       |     "tool_name": "get_document_text"
  },                                       |   },
  {                                        |   {
    "tool

インストラクションを変更して、`log_policy_audit` を使用するタイミングを変更します。

In [25]:
instruction='''
あなたは人事アシスタントです。
1. list_policy_documents を使用してドキュメントIDを検索してください。
2. get_document_text を使用して内容を取得してください。
3. ドキュメントのテキストに基づいてユーザーの質問に答えてください。

markdownを使用せずに、最終回答のみを日本語で出力します。

**重要**
- ドキュメントを読む前に、監査のために必ず log_policy_audit を使用してアクセスを記録すること。
'''

EXPERIMENT_RUN = f'trajectory-{get_id()}'

trajectory_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + '/multiple-metric-eval',
)

chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.0))
trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-trajectory-7cda40d6 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/multiple-metric-eval/eval_results_2026-01-10-05-21-20-7c3bf.csv'}


100%|██████████| 2/2 [00:09<00:00,  4.80s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 10/10 [00:00<00:00, 10.46it/s]

All 10 metric requests are successfully computed.
Evaluation Took:0.9783810859998994 seconds


In [26]:
df = trajectory_eval_result.metrics_table[[
    'prompt',
    'trajectory_exact_match/score',
    'trajectory_in_order_match/score',
    'trajectory_any_order_match/score',
]]

display(Markdown(df.to_markdown(index=False)))

| prompt                                                                                 |   trajectory_exact_match/score |   trajectory_in_order_match/score |   trajectory_any_order_match/score |
|:---------------------------------------------------------------------------------------|-------------------------------:|----------------------------------:|-----------------------------------:|
| 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。   |                              0 |                                 0 |                                  1 |
| 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。 |                              0 |                                 0 |                                  1 |

In [27]:
df = trajectory_eval_result.metrics_table[[
    'prompt',
    'trajectory_precision/score',
    'trajectory_recall/score',
]]
df['trajectory_precision/score'] = df['trajectory_precision/score'].map('{:.2f}'.format)
df['trajectory_recall/score'] = df['trajectory_recall/score'].map('{:.2f}'.format)

display(Markdown(df.to_markdown(index=False)))

| prompt                                                                                 |   trajectory_precision/score |   trajectory_recall/score |
|:---------------------------------------------------------------------------------------|-----------------------------:|--------------------------:|
| 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。   |                            1 |                         1 |
| 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。 |                            1 |                         1 |

- いずれのクエリーも `trajectory_in_order_match` が 0 なので、正解データとはツール使用の順序が異なることがわかります。
- いずれのクエリーも `trajectory_any_order_match` が 1 なので、使用順序を無視すれば、正解データに含まれるツール使用はすべて含まれることがわかります。
- `trajectory_precision` が 1 なので、正解データに含まれないツール使用はなかったことがわかります。

正解データのトラジェクトリ（Reference Trajectory）と再実行時のトラジェクトリ（Predicted Trajectory）を表示して確認します。

In [28]:
for c, item in trajectory_eval_result.metrics_table.iterrows():
    print('\n== Query ==')
    print(item.prompt + '\n')
    reference = '========== Reference Trajectory ==========\n' 
    reference += json.dumps(item.reference_trajectory, indent=2, sort_keys=True)
    predicted = '========== Predicted Trajectory ==========\n' 
    predicted += json.dumps(item.predicted_trajectory, indent=2, sort_keys=True)
    print(merge_texts_side_by_side(reference, predicted, width=42))


== Query ==
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。

========== Reference Trajectory ========== | ========== Predicted Trajectory ==========
[                                          | [
  {                                        |   {
    "tool_input": {                        |     "tool_input": {
      "category": "benefits"               |       "category": "benefits"
    },                                     |     },
    "tool_name": "list_policy_documents"   |     "tool_name": "list_policy_documents"
  },                                       |   },
  {                                        |   {
    "tool_input": {                        |     "tool_input": {
      "doc_id": "ben_med_gold"             |       "doc_id": "ben_med_gold",
    },                                     |       "reason": "personal"
    "tool_name": "get_document_text"       |     },
  },                                       |     "tool_name": "log_policy_audit"
  {                                 